In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from tqdm import tqdm_notebook as tn
import time
import schedule

# 신고국가(reporter)와 그 무역상대국가(partner)

In [164]:
# url='https://comtrade.un.org/api/get?max=500&type=C&freq=A&px=HS&ps=2018&r=152&p=all&rg=all&cc=851712'
url = 'https://comtrade.un.org/api/get?r=all&px=HS&freq=A&ps=2021&p=410&rg=1&cc=1702&max=10000&type=C'
un_data=requests.get(url)
print(un_data.content)

c:\Users\signlab026\anaconda3\envs\gibo\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


b'U\x00S\x00A\x00G\x00E\x00 \x00L\x00I\x00M\x00I\x00T\x00:\x00 \x00H\x00o\x00u\x00r\x00l\x00y\x00 \x00u\x00s\x00a\x00g\x00e\x00 \x00l\x00i\x00m\x00i\x00t\x00 \x00o\x00f\x00 \x001\x000\x000\x00 \x00a\x00c\x00t\x00i\x00o\x00n\x00s\x00 \x00r\x00e\x00a\x00c\x00h\x00e\x00d\x00.\x00 \x00Y\x00o\x00u\x00 \x00m\x00a\x00y\x00 \x00r\x00e\x00s\x00u\x00m\x00e\x00 \x00u\x00s\x00e\x00 \x00o\x00f\x00 \x00t\x00h\x00i\x00s\x00 \x00s\x00e\x00r\x00v\x00i\x00c\x00e\x00 \x00a\x00t\x00 \x002\x000\x002\x002\x00-\x000\x008\x00-\x002\x003\x00T\x000\x006\x00:\x001\x002\x00:\x005\x008\x00Z\x00.\x00'


In [163]:
url = 'https://comtrade.un.org/api/get?r=all&px=HS&freq=A&ps=2021&p=410&rg=1&cc=1702&max=10000&type=C&fmt=csv'
comtrade = pd.read_csv(url)
comtrade


HTTPError: HTTP Error 409: Conflict

In [93]:
comtrade

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Qty,Alt Qty Unit Code,Alt Qty Unit,Alt Qty,Netweight (kg),Gross weight (kg),Trade Value (US$),CIF Trade Value (US$),FOB Trade Value (US$),Flag
0,H5,2017,2017,2017,4,0,1,Import,12,Algeria,...,4563,NaN,NaN,NaN,4563.0,NaN,21174,NaN,NaN,0
1,H5,2017,2017,2017,4,0,1,Import,36,Australia,...,1254504,NaN,NaN,NaN,1671375.0,NaN,6316862,NaN,NaN,6
2,H5,2017,2017,2017,4,0,2,Export,36,Australia,...,1415846,NaN,NaN,NaN,1448365.0,NaN,11360895,NaN,NaN,0
3,H5,2017,2017,2017,4,0,1,Import,40,Austria,...,64857,NaN,NaN,NaN,64857.0,NaN,314330,NaN,NaN,0
4,H5,2017,2017,2017,4,0,2,Export,40,Austria,...,36214,NaN,NaN,NaN,36214.0,NaN,993099,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,H3,2018,2018,2018,4,0,1,Import,740,Suriname,...,12300,NaN,NaN,NaN,12300.0,NaN,11501,NaN,NaN,0
895,H3,2019,2019,2019,4,0,1,Import,740,Suriname,...,11246,NaN,NaN,NaN,11246.0,NaN,20062,NaN,NaN,0
896,H3,2020,2020,2020,4,0,1,Import,740,Suriname,...,8614,NaN,NaN,NaN,8614.0,NaN,23089,NaN,NaN,0
897,H3,2021,2021,2021,4,0,1,Import,740,Suriname,...,5664,NaN,NaN,NaN,5664.0,NaN,20272,NaN,NaN,0


In [45]:
comtrade.to_csv('test.csv')

# Get Data

In [160]:

class Un_Comtrade_GetData(): 
    def __init__(self, r, px ,ps, p, rg, cc, freq, g_max, g_type, fmt): 
        self.r = r # 무역 보고자 / reporter
        self.px = px # 분류 / HS 고정 
        self.freq = freq # 시간1 / A = 연간, C = 월간
        self.ps = ps # 시간2 / YYYY or YYYYMM
        self.p = p # 무역 상대국가 / partner
        self.rg = rg # 수입-수출 분류 / import(수입), export(수출), re-import(재수입), re-import(재수출)
        self.cc = cc # HS 코드 
        self.g_max = g_max # 긁어오는 한도 / 인증 토큰 없으면 10000개가 한계
        self.g_type = g_type # 무역 종류 / C = 상품, S = 서비스
        self.fmt = fmt # csv or json
        
    def get_data(self):
        data_list = []
        for hs in tn(self.cc): # HS코드
            for rp in self.r: # reporter 나라
                for y in self.ps: # 연도
                        for ie in self.rg: # 수입 수출
                            try:
                                print('cc: ', hs)
                                print('ps: ', y)
                                print('r: ', rp)
                                print('rg: ', ie)
                                url = f'https://comtrade.un.org/api/get?r={rp}&px={self.px}&freq={self.freq}&ps={y}&p={self.p}&rg={ie}&cc={hs}&max={self.g_max}&type={self.g_type}&fmt={self.fmt}'
                                print('url :', url)
                                comtrade = pd.read_csv(url)
                                data_list.append(comtrade)
                            
                            except:
                                print(hs, '코드 ', ie, '(1:수입, 2:수출) ',rp, '나라 ',  y, '연도에서 에러 발생')
                                pass
                
            time.sleep(3600)
                    
        
        return pd.concat(data_list)
    

    
  

In [161]:
# r, px ,ps, p, rg, cc, freq, g_max, g_type, fmt
food = Un_Comtrade_GetData(
    r = 'all',
    px = 'HS',
    ps = (2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012),
    p = 'all',
    rg = (1, 2),
    cc = (1601, 1602, 1603, 1604, 1605, 1901, 1902, 1905, 2004, 2005, 2103, 2104, 2105, 2106,
          1518, 1702, 1806, 2001, 2202, 1518, 1704, 1904, 2101, 2208),
    freq= 'A',
    g_max = '10000',
    g_type = 'C',
    fmt = 'csv' # 판다스만됨
)



In [162]:
data = get.get_data()

C:\Users\signlab026\AppData\Local\Temp\ipykernel_17268\52439790.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for hs in tn(self.cc):


  0%|          | 0/24 [00:00<?, ?it/s]

cc : 1601
ps : 2021
rg : 1
url : https://comtrade.un.org/api/get?r=all&px=HS&freq=A&ps=2021&p=410&rg=1&cc=1601&max=10000&type=C&fmt=csv
cc : 1601
ps : 2021
rg : 2
url : https://comtrade.un.org/api/get?r=all&px=HS&freq=A&ps=2021&p=410&rg=2&cc=1601&max=10000&type=C&fmt=csv
cc : 1601
ps : 2020
rg : 1
url : https://comtrade.un.org/api/get?r=all&px=HS&freq=A&ps=2020&p=410&rg=1&cc=1601&max=10000&type=C&fmt=csv
cc : 1601
ps : 2020
rg : 2
url : https://comtrade.un.org/api/get?r=all&px=HS&freq=A&ps=2020&p=410&rg=2&cc=1601&max=10000&type=C&fmt=csv
cc : 1601
ps : 2019
rg : 1
url : https://comtrade.un.org/api/get?r=all&px=HS&freq=A&ps=2019&p=410&rg=1&cc=1601&max=10000&type=C&fmt=csv
cc : 1601
ps : 2019
rg : 2
url : https://comtrade.un.org/api/get?r=all&px=HS&freq=A&ps=2019&p=410&rg=2&cc=1601&max=10000&type=C&fmt=csv
cc : 1601
ps : 2018
rg : 1
url : https://comtrade.un.org/api/get?r=all&px=HS&freq=A&ps=2018&p=410&rg=1&cc=1601&max=10000&type=C&fmt=csv
cc : 1601
ps : 2018
rg : 2
url : https://comtrad

In [170]:
data.to_csv('테스트2.csv')